# II. Tiền xử lý dữ liệu

Trong phần này, chúng ta sẽ thực hiện các bước chuẩn bị dữ liệu trước khi đưa vào mô hình:  
- Đọc dữ liệu  
- Loại bỏ cột không cần thiết  
- Xử lý giá trị thiếu  
- Tạo biến thời gian  
- Tách target và feature  
- Mã hoá biến phân loại  
- Chuẩn hóa dữ liệu số  
- (Tuỳ chọn) Giảm chiều với PCA  
- Kiểm tra kết quả  


### 1. Import thư viện cần thiết


In [207]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

### 2. Đọc dữ liệu
- Load file **dataset.csv** vào DataFrame `df`.

In [208]:
df = pd.read_csv('dataset.csv')
print(f"Đọc dữ liệu thành công! Kích thước: {df.shape[0]} dòng x {df.shape[1]} cột.")
df.head()

Đọc dữ liệu thành công! Kích thước: 200000 dòng x 47 cột.


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,0,11/15/2014,236000,2.0,,nochg,2025,47.2917,-122.3658,53,...,0,0,0,0,0,0,0,0,0,I
1,1,1/15/1999,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,...,0,0,0,0,0,1,0,0,0,Q
2,2,8/15/2006,341000,1.0,,nochg,2025,47.4733,-122.1901,30,...,0,0,0,0,0,0,0,0,0,K
3,3,12/15/1999,267000,1.0,,nochg,2025,47.4739,-122.3295,96,...,0,0,0,0,0,0,0,0,0,G
4,4,7/15/2018,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,...,0,0,0,0,0,0,0,0,0,P


### 3. Loại bỏ cột không cần thiết

Dựa trên kết quả EDA, các cột sau không mang thông tin dự đoán hoặc quá đồng nhất, nên chúng ta loại bỏ:

- **`id`**  
  - Là khóa định danh duy nhất cho mỗi bản ghi.  
  - Mỗi giá trị chỉ xuất hiện 1 lần, không có tính lặp lại.  
  - Không chứa thông tin liên quan đến **giá bán**.

- **`sale_warning`**  
  - Trong **200 000** bản ghi, có tới **180 689** giá trị trống (~90%).  
  - Các giá trị còn lại phân tán, không thấy xu hướng rõ ràng với `sale_price`.

- **`join_status`**  
  - Có 8 giá trị, nhưng  
    - `"nochg"` chiếm **126 281/200 000** ≈ 63%  
    - Các trạng thái còn lại rất ít xuất hiện, không đủ dữ liệu để rút ra mối liên hệ với giá.

- **`join_year`**  
  - 75% bản ghi có giá trị **2025** (median = 2025, IQR cao)  
  - Chỉ có ~25% trải từ 1999–2024, quá đồng nhất để làm feature hữu ích.

In [209]:
drop_cols = ['id','sale_warning','join_status','join_year']
df = df.drop(columns=drop_cols)
print(f"Loại bỏ cột thành công! Còn lại {df.shape[1]} cột.")

Loại bỏ cột thành công! Còn lại 43 cột.


### 4. Xử lý giá trị thiếu
- `sale_nbr` 21% missing → dùng median vì phân phối lệch vừa (Skewness 1.16).
- `subdivision` (8.8%) và `submarket` (0.9%) thiếu → điền 'Unknown' giữ nguyên tần suất và tránh tạo quá nhiều giá trị mới.

In [210]:
df['sale_nbr']    = df['sale_nbr'].fillna(df['sale_nbr'].median())
df['subdivision'] = df['subdivision'].fillna('Unknown')
df['submarket']   = df['submarket'].fillna('Unknown')

# Kiểm tra còn missing
miss = df[['sale_nbr','subdivision','submarket']].isnull().sum()
print(f"Xử lý missing thành công! Số giá trị thiếu:\n{miss}")

Xử lý missing thành công! Số giá trị thiếu:
sale_nbr       0
subdivision    0
submarket      0
dtype: int64


### 5. Gộp các category hiếm thành “Other”

Trong bước EDA, ta đã quan sát:

- **`subdivision`** có **10.376** giá trị khác nhau, nhưng **100 %** trong số đó xuất hiện với tần suất **dưới 1 %** mỗi loại  
  - Ví dụ: giá trị phổ biến nhất là `MAPLE LEAF TO GREEN LAKE CIRCLE POR OF` cũng chỉ chiếm khoảng 723/200 000 ≈ **0,36 %**  
- **`submarket`** có **19** giá trị, nhưng chỉ **5** giá trị (`K`, `B`, `I`, `R`, `Q`) chiếm **≥ 1 %** dữ liệu; các giá trị còn lại đều **< 1 %**

**Vấn đề nếu One-Hot toàn bộ**:  
- Sẽ sinh hơn **10.000** cột dummy cho `subdivision` và nhiều cột cho `submarket`  
- Gây **tốn tài nguyên**, **chậm huấn luyện**, dễ **overfitting** do quá nhiều biến thưa

**Giải pháp**:  
- Gộp tất cả các giá trị có tần suất **< 1 %** thành nhãn **`Other`**  
- Chỉ giữ lại những giá trị **phổ biến (≥ 1 %)**  
- Kết quả:
  - Giảm mạnh số cột One-Hot  
  - Vẫn bảo toàn phân bố các giá trị chính  



In [211]:
# Gộp subdivision và submarket hiếm (<1%) thành 'Other'
for col in ['subdivision','submarket']:
    freq = df[col].value_counts(normalize=True)
    top = freq[freq >= 0.01].index
    df[col] = df[col].where(df[col].isin(top), 'Other')

print("Gộp category hiếm xong!")

Gộp category hiếm xong!


### 5. Tạo biến thời gian từ `sale_date`

Trong EDA, chúng ta không sử dụng trực tiếp chuỗi `sale_date` (ví dụ `"11/15/2014"`) vì:

- Dữ liệu dạng text khó so sánh và không thể sử dụng trực tiếp cho mô hình.  
- Xu hướng giá theo **năm** và **tháng** quan trọng hơn: ví dụ, thị trường có thể lên xuống định kỳ theo mùa hoặc theo năm.

**Giải pháp**:

1. **Chuyển `sale_date` sang kiểu datetime** để thao tác dễ dàng.  
2. **Trích năm (`sale_year`) và tháng (`sale_month`)** để model học được xu hướng biến động theo thời gian.  
3. **Tạo biến `decade`** từ `year_built` (năm xây dựng) bằng cách nhóm theo thập niên.  
   - EDA cho thấy giá nhà thường phụ thuộc vào tuổi của ngôi nhà trong các nhóm 10 năm (ví dụ: 1970s, 1980s, …).  
   - Nhóm thập niên giúp giảm nhiễu so với giữ nguyên từng năm.

In [212]:
df['sale_date']  = pd.to_datetime(df['sale_date'])
df['sale_year']  = df['sale_date'].dt.year
df['sale_month'] = df['sale_date'].dt.month
df['decade']     = (df['year_built'] // 10) * 10
print(f"Bước 5: Tạo biến thời gian & decade xong! sale_year={df.loc[0,'sale_year']}, decade={df.loc[0,'decade']}")

Bước 5: Tạo biến thời gian & decade xong! sale_year=2014, decade=1970


### 6. Tách target và feature
- `y` là cột `sale_price`  
- `X` là toàn bộ các cột còn lại, ngoại trừ `sale_price` và cột gốc `sale_date`

In [213]:
y = df['sale_price']
X = df.drop(columns=['sale_price','sale_date'])
print(f"Tách X, y thành công! X có {X.shape[1]} cột, y có {y.shape[0]} giá trị.")

Tách X, y thành công! X có 44 cột, y có 200000 giá trị.


### 7. Mã hoá biến phân loại (One-Hot Encoding)
- Áp dụng với `city`, `zoning`, `subdivision`, `submarket`


In [214]:
categorical_cols = ['city','zoning','subdivision','submarket']
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
print(f"One-Hot Encoding thành công! X bây giờ có {X.shape[1]} cột.")

One-Hot Encoding thành công! X bây giờ có 598 cột.


In [215]:
# Lấy danh sách tất cả các cột
columns = X.columns.tolist()

# In tên 20 cột đầu
print("Danh sách 20 cột đầu sau One-Hot Encoding:")
print(", ".join(columns[:20]))

Danh sách 20 cột đầu sau One-Hot Encoding:
sale_nbr, latitude, longitude, area, present_use, land_val, imp_val, year_built, year_reno, sqft_lot, sqft, sqft_1, sqft_fbsmt, grade, fbsmt_grade, condition, stories, beds, bath_full, bath_3qtr


### 8. Loại bỏ dummy variance thấp
Sau khi One-Hot, chúng ta có **~598** cột giả (dummy). Tuy nhiên nhiều cột chỉ xuất hiện trong **dưới 1%** hoặc **trên 99%** bản ghi, tức là gần như **luôn 0** hoặc **luôn 1**. Những cột này có **phương sai rất thấp** (p⋅(1−p) < 0.01) nên **không mang thêm thông tin** để dự đoán và chỉ làm tăng độ phức tạp.

**Lý do chi tiết**:  
- Với một cột dummy, xác suất xuất hiện p ≈ 0.01 → variance = p(1−p) ≈ 0.0099 < 0.01  
- Tương tự p ≈ 0.99 → variance ≈ 0.0099 < 0.01  
- Những cột này gần như không thay đổi, không giúp phân biệt các mẫu.

**Giải pháp**: dùng `VarianceThreshold(threshold=0.01)` để tự động loại bỏ các cột có variance < 0.01.


In [216]:
# Giữ dummy có variance ≥0.01 (~xuất hiện trong 1% bản ghi)
selector_var = VarianceThreshold(threshold=0.01)
X_var = selector_var.fit_transform(X[dummy_cols])

# 3. Lấy tên dummy giữ lại
kept_dummy_var = selector_var.get_feature_names_out(dummy_cols)
print(f"Sau VarianceThreshold, giữ lại {len(kept_dummy_var)}/{len(dummy_cols)} cột dummy")

# 4. Đưa về DataFrame để dễ thao tác
X_var = pd.DataFrame(
    X_var,
    columns=kept_dummy_var,
    index=X.index
)

Sau VarianceThreshold, giữ lại 64/558 cột dummy


### 9. Loại bỏ dummy tương quan thấp với target
- Sau khi đã loại bỏ những cột dummy có **variance thấp** (xuất hiện dưới 1% hoặc trên 99% bản ghi), chúng ta cần tiếp tục loại bỏ những cột dummy **không mang thông tin** dự đoán.

- EDA cho thấy hầu hết các biến giả này có **hệ số tương quan Pearson** với `sale_price` rất nhỏ (|r| < 0.01), nghĩa là gần như không liên quan. Việc giữ lại chúng chỉ làm tăng độ phức tạp và dễ dẫn đến **overfitting**.

- **Hệ số tương quan Pearson (r)** đo mức độ mối liên hệ tuyến tính giữa hai biến (−1 đến +1).  
- Khi **|r| < 0.01**, biến gần như không giúp mô hình cải thiện dự đoán, nên ta **loại bỏ**.

In [217]:
# 1. Tính tương quan tuyệt đối giữa dummy và sale_price
corrs = X_var.apply(lambda col: col.corr(y)).abs()

# 2. Chọn dummy có |corr| ≥ 0.01
keep_dummy_corr = corrs[corrs >= 0.01].index.tolist()
print(f"Sau lọc corr ≥0.01, giữ lại {len(keep_dummy_corr)}/{len(kept_dummy_var)} dummy columns")

# 3. DataFrame dummy cuối cùng
X_dummy_sel  = X_var[keep_dummy_corr]

Sau lọc corr ≥0.01, giữ lại 57/64 dummy columns


In [218]:
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Giả sử numeric_cols_sel là danh sách numeric cuối cùng sau bước cập nhật
X_numeric = X[numeric_cols]

# Ghép lại
X_sel = pd.concat([X_numeric, X_dummy_sel], axis=1)
print("Kết hợp numeric + dummy xong! Kích thước X_sel:", X_sel.shape)
X_sel.head()

Kết hợp numeric + dummy xong! Kích thước X_sel: (200000, 97)


,sale_nbr,latitude,longitude,area,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,...,submarket_J,submarket_K,submarket_L,submarket_M,submarket_O,submarket_Other,submarket_P,submarket_Q,submarket_R,submarket_S
0,2.0,47.2917,-122.3658,53,2,167000,372000,1975,0,10919,...,False,False,False,False,False,False,False,False,False,False
1,2.0,47.6531,-122.1996,74,2,1184000,598000,1962,0,8900,...,False,False,False,False,False,False,False,True,False,False
2,1.0,47.4733,-122.1901,30,2,230000,356000,1986,0,4953,...,False,True,False,False,False,False,False,False,False,False
3,1.0,47.4739,-122.3295,96,2,190000,518000,1998,0,6799,...,False,False,False,False,False,False,False,False,False,False
4,2.0,47.7516,-122.1222,36,2,616000,1917000,1998,0,31687,...,False,False,False,False,False,False,True,False,False,False


### 10. Chuẩn hóa dữ liệu số

Mỗi biến số (ví dụ `land_val`, `sqft`, `beds`, …) có thang đo và phạm vi khác nhau rất lớn. Ví dụ:

- `land_val` có giá trị trung bình khoảng **460 000** và độ lệch chuẩn **351 000**  
- Trong khi `beds` chỉ dao động từ **0** đến **14**

Nếu để nguyên, thuật toán sẽ “ưu ái” biến có giá trị lớn hơn khi tính khoảng cách hay gradient, dẫn đến kết quả **không công bằng** giữa các biến.

**StandardScaler** sẽ chuyển mỗi biến numeric về phân phối chuẩn với:  
- **Mean (trung bình)** = 0  
- **Std (độ lệch chuẩn)** = 1  

Cụ thể, mỗi giá trị \(x\) được biến thành:  
$$
x' = \frac{x - \mu}{\sigma}
$$


In [219]:
# Lấy sub-DataFrame các cột numeric còn giữ
X_num_sel = X_sel[numeric_cols]

scaler = StandardScaler()
scaled_array = scaler.fit_transform(X_num_sel)

# 2. Chuyển thành DataFrame, giữ tên cột và index gốc
X_num_scaled = pd.DataFrame(
    scaled_array,
    columns = X_num_sel.columns,
    index   = X_num_sel.index
)

# 3. Kiểm tra
print("Bước 10: Chuẩn hóa dữ liệu số thành công! Kích thước:", X_num_scaled.shape)
X_num_scaled.head()

Bước 10: Chuẩn hóa dữ liệu số thành công! Kích thước: (200000, 40)


,sale_nbr,latitude,longitude,area,present_use,land_val,imp_val,year_built,year_reno,sqft_lot,...,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,sale_year,sale_month,decade
0,-0.129472,-1.804697,-1.107207,0.160541,-0.292926,-0.835136,-0.325422,0.026690,-0.175252,-0.075507,...,-0.29763,-0.082718,-0.14622,-0.141590,-0.070396,-0.083843,-0.074277,0.370359,1.386632,0.016085
1,-0.129472,0.727714,0.077070,0.934537,-0.292926,2.061940,0.288626,-0.398920,-0.175252,-0.128735,...,-0.29763,-0.082718,-0.14622,2.685961,-0.070396,-0.083843,-0.074277,-1.604494,-1.783976,-0.310683
2,-1.138568,-0.532185,0.144763,-0.687169,-0.292926,-0.655671,-0.368895,0.386822,-0.175252,-0.232791,...,-0.29763,-0.082718,-0.14622,-0.141590,-0.070396,-0.083843,-0.074277,-0.682896,0.435450,0.342853
3,-1.138568,-0.527981,-0.848547,1.745389,-0.292926,-0.769617,0.071264,0.779694,-0.175252,-0.184124,...,-0.29763,-0.082718,-0.14622,-0.141590,-0.070396,-0.083843,-0.074277,-1.604494,1.703693,0.669621
4,-0.129472,1.417926,0.628592,-0.466027,-0.292926,0.443908,3.872382,0.779694,-0.175252,0.472007,...,-0.29763,-0.082718,-0.14622,-0.141590,-0.070396,-0.083843,-0.074277,0.896986,0.118389,0.669621


### 11. Giảm chiều với PCA và ghép với categorical
- **Mục đích**:  
    - Dữ liệu số sau chuẩn hóa còn **40 chiều**.  
    - Áp dụng PCA để gom giữ **95% tổng phương sai**, giảm xuống còn **30 thành phần chính**.  
    - Ghép `X_reduced` (numeric) với phần dummy categorical đã lọc (`X_cat_sel`) để có bộ feature cuối cùng `X_final` với **87 cột**.

- **Lý do chi tiết**:  
  - **PCA** tự động xác định tổ hợp tuyến tính các biến numeric để tối ưu hóa phương sai giải thích.
  - Dùng mức 95% để giữ lại hầu hết tín hiệu nhưng vẫn giảm đáng kể số chiều.
  - Kết quả PCA giúp giảm noise và giảm số chiều, tăng tốc độ huấn luyện.  
  - Các dummy categorical quan trọng vẫn được giữ nguyên để model học được thông tin phân loại như `city`, `zoning`.

In [220]:
pca = PCA(n_components=0.95, random_state=42)
X_reduced = pca.fit_transform(X_num_scaled)
print(f"Bước 11: PCA thành công! Số chiều gốc = {X_num_scaled.shape[1]}, sau PCA = {X_reduced.shape[1]}")

Bước 11: PCA thành công! Số chiều gốc = 40, sau PCA = 30


In [221]:
# Giả sử muốn giữ lại categorical dummy đã lọc trong X_sel nhưng không scale/PCA
X_cat_sel = X_sel.drop(columns=numeric_cols).values

# Ghép
X_final = np.hstack([X_reduced, X_cat_sel])
print("Ghép numeric+categorical xong! Kích thước X_final:", X_final.shape)

Ghép numeric+categorical xong! Kích thước X_final: (200000, 87)


### 12. Kiểm tra kích thước cuối cùng

In [222]:
print("Kích thước X gốc sau One-Hot và lọc:", X_sel.shape)
print("Kích thước X_reduced (numeric PCA):", X_reduced.shape)
print("Kích thước X_final (numeric PCA + categorical):", X_final.shape)
print("Kích thước y:", y.shape)
print("Hoàn thành toàn bộ tiền xử lý dữ liệu!")

Kích thước X gốc sau One-Hot và lọc: (200000, 97)
Kích thước X_reduced (numeric PCA): (200000, 30)
Kích thước X_final (numeric PCA + categorical): (200000, 87)
Kích thước y: (200000,)
Hoàn thành toàn bộ tiền xử lý dữ liệu!


In [223]:
# 1. Chuyển X_reduced (numpy array) về DataFrame với tên cột PC1, PC2, …
pca_cols = [f"PC{i+1}" for i in range(X_reduced.shape[1])]
X_reduced_df = pd.DataFrame(X_reduced, columns=pca_cols, index=X_sel.index)

# 2. Lấy phần categorical dummy đã lọc dưới dạng DataFrame
X_cat_sel_df = X_sel.drop(columns=numeric_cols).copy()

# 3. Ghép hai DataFrame lại với nhau
X_final_df = pd.concat([X_reduced_df, X_cat_sel_df], axis=1)

# Kết quả
X_final_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,submarket_J,submarket_K,submarket_L,submarket_M,submarket_O,submarket_Other,submarket_P,submarket_Q,submarket_R,submarket_S
0,-1.503543,-0.585453,1.715450,-0.064378,-0.178853,-1.497020,0.357814,-0.259905,0.703341,0.527011,...,False,False,False,False,False,False,False,False,False,False
1,-0.483306,2.162351,0.267204,-0.705821,0.398560,1.166909,0.922957,1.228301,1.278198,-0.900333,...,False,False,False,False,False,False,False,True,False,False
2,-0.029303,-2.119967,-0.359337,0.962393,-0.436443,1.050526,0.880459,0.017190,-0.860492,-0.132316,...,False,True,False,False,False,False,False,False,False,False
3,1.432736,-0.829134,-0.616422,-0.167648,-0.210156,0.685293,1.742891,-0.045558,-1.438272,-0.333267,...,False,False,False,False,False,False,False,False,False,False
4,6.332717,0.536902,1.654519,-1.291845,-0.776623,1.227724,-1.444977,-0.635197,1.210476,1.856399,...,False,False,False,False,False,False,True,False,False,False
